# Ausgleichungsrechnung

In [22]:
import numpy as np
from sympy import *
import math

## Formeln

In [23]:
sym = [
    Symbol('a'),
    Symbol('b') 
]

formeln = [
    sym[0],
    sym[1],
    sqrt(sym[0]**2 + sym[1]**2),
    sym[1]+sym[0]
]

## Messwerte

In [24]:
l  = [
    2.99,
    4.01,
    5.02,
    6
]

In [25]:
x0 = [
    2,
    5
]

## Gewicht

In [26]:
P = np.eye(len(l))
print('P',P)

P [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


## Iterative Berechnung

In [27]:
for durchlauf in range (10):
    A = np.empty((len(formeln),len(sym)))
    for f in range(len(formeln)):
        for s in range(len(sym)):
            formel = formeln[f].diff(sym[s])
            for sy in range(len(sym)):
                formel = formel.subs(sym[sy], x0[sy])
            A[f][s] = formel


    l0 = np.empty(len(l))
    for i in range(len(l0)):
        formel = formeln[i]
        for sy in range(len(sym)):
            formel = formel.subs(sym[sy], x0[sy])
        l0[i] = formel
    dl = l - l0

    N = np.matmul(np.matmul(np.transpose(A),P),A)
    h = np.matmul(np.matmul(np.transpose(A),P),dl)
    Qx = np.linalg.inv(N)
    dx = np.matmul(Qx,h)
    print(durchlauf, dx)
    x0 = x0 + dx

    if np.linalg.norm(dx) < 0.0001:
        break
print('x0',x0)

0 [ 0.62997178 -1.10977425]
1 [ 0.06152002 -0.0826079 ]
2 [ 0.00549425 -0.00493452]
3 [ 0.0004197  -0.00035566]
4 [ 3.13962078e-05 -2.64869521e-05]
x0 [2.69743715 3.80230118]


## Genauigkeit

In [28]:
Va = np.matmul(A, dx) - dl

z = len(l)
s = len(x0)

s0 = math.sqrt(np.matmul(np.matmul(np.transpose(Va), P), Va) / (z - s))
print('s0',s0)
Sx = np.sqrt(np.diag(Qx)) * s0
print('Sx',Sx)


s0 0.5033284013034892
Sx [0.40799961 0.38187052]


### Redundanzanteile

In [29]:
Red = np.eye(len(l)) - np.matmul(np.matmul(np.matmul(A,Qx),np.transpose(A)),P)
f = np.trace(Red)
k = 1.96
r = np.diag(Red)
print('r', r)


r [0.34292245 0.42438859 0.73961338 0.49307558]


### Grober Fehler

In [30]:
Ql = np.linalg.inv(P)
Qv = np.matmul(np.matmul(Red, Ql),np.transpose(Red))
Sv = s0 * np.sqrt(np.diag(Qv))
GF = -Va/np.diag(Red)
print('GF',GF)

GF [ 0.8531458   0.48940719  0.48412116 -1.01351261]


### GRZW

In [31]:
GRZW = Sv/np.diag(Red)*k
print ('GRZW', GRZW)

GRZW [1.68464946 1.5143479  1.14711017 1.4049173 ]


### Normierte Verbesserung

In [32]:
NV = np.abs(Va) / Sv
print('NV',NV)

NV [0.99258974 0.6334331  0.82718949 1.41395136]
